In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf   # only for seeding; can remove if you want

# ==========================================================
# Reproducibility
# ==========================================================
def set_seed(seed=42):
    tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ==========================================================
# Sliding Window: use past T days to predict next day
# ==========================================================
def build_sliding_window(X, y, T):
    """
    X: (N, F)
    y: (N,)
    Returns:
        X_seq: (N-T, T, F)
        y_seq: (N-T, 1)
    """
    X_seq, y_seq = [], []
    for i in range(T, len(X)):
        X_seq.append(X[i-T:i])    # past T days as one sequence
        y_seq.append(y[i])        # predict value at day i
    X_seq = torch.tensor(np.array(X_seq), dtype=torch.float32)
    y_seq = torch.tensor(np.array(y_seq), dtype=torch.float32).view(-1, 1)
    return X_seq, y_seq


# ==========================================================
# Attention Module (over time steps)
# ==========================================================
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        # simple linear scoring over time dimension
        self.attention_weights = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, lstm_output):
        """
        lstm_output: (batch_size, seq_length, hidden_dim)
        Returns:
            context_vector: (batch_size, hidden_dim)
            attn_weights:   (batch_size, seq_length, 1)
        """
        scores = self.attention_weights(lstm_output).squeeze(-1)   # (B, T)
        alpha = torch.softmax(scores, dim=1).unsqueeze(-1)         # (B, T, 1)
        context = (lstm_output * alpha).sum(dim=1)                 # (B, hidden_dim)
        return context, alpha


# ==========================================================
# CNN-BiLSTM-Attention Model (proper sequence model)
# ==========================================================
class CNN_BiLSTM_AM_Model(nn.Module):
    def __init__(self, num_features, lstm_hidden_size=64, conv_filters=64):
        """
        num_features: number of input features per day (F)
        """
        super().__init__()

        # 1D CNN over time, treating features as channels
        # Input to conv1: (B, F, T)
        self.conv1 = nn.Conv1d(in_channels=num_features,
                               out_channels=conv_filters,
                               kernel_size=1,
                               padding=0)
        self.conv_activation = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=1, stride=1, padding=0)

        # BiLSTM for temporal dependencies
        self.lstm = nn.LSTM(input_size=conv_filters,
                            hidden_size=lstm_hidden_size,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)

        # Attention over BiLSTM outputs
        self.attention = Attention(hidden_dim=lstm_hidden_size * 2)

        # Fully connected output layer
        self.output_layer = nn.Linear(lstm_hidden_size * 2, 1)

    def forward(self, x):
        """
        x: (batch_size, seq_length=T, num_features=F)
        """
        # CNN expects (B, C_in, T) where C_in = num_features
        x = x.permute(0, 2, 1)  # (B, F, T)

        x = self.conv_activation(self.conv1(x))   # (B, conv_filters, T)
        x = self.pool(x)                          # (B, conv_filters, T)

        # Back to (B, T, conv_filters) for LSTM
        x = x.permute(0, 2, 1)  # (B, T, conv_filters)

        # BiLSTM
        lstm_out, _ = self.lstm(x)  # (B, T, 2*lstm_hidden_size)

        # Attention
        context_vector, attn_weights = self.attention(lstm_out)  # (B, 2*h), (B, T, 1)

        # Output
        output = self.output_layer(context_vector)  # (B, 1)
        return output


# ==========================================================
# Main: multi-seed training, metrics, CSV
# ==========================================================
symbol_list = ['AAPL','META','MSFT','AMZN','GOOG','NVDA','F','TGT','EL']
#symbol_list = ['F','TGT','EL']
seeds = [0, 1, 2, 3]
steps_list = [2,5,10]



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
    
for steps in steps_list:
    for symbol in symbol_list:
    

        re_train_error = []
        re_train_acc = []
        re_testing_error = []
        re_testing_acc = []
        for seed in seeds:
            print("\n=======================================")
            print(f" Seed = {seed}")
            print("=======================================\n")

            set_seed(seed)


            # -----------------------------
            # Load and preprocess data
            # -----------------------------
            data = pd.read_csv(f'../../Datasets/data{symbol}.csv')

            # set index
            data.index = data['Date']
            data.drop(columns=['Date'], inplace=True)

            # remove columns with infinity
            columns_infinity = data.columns.to_series()[np.isinf(data).any()]
            data.drop(columns=columns_infinity, inplace=True)

            # keep Close + features (your original order)
            data = data[['Close', 'pct_change',
                         'log_change', '7sma', '14sma', '21sma', '7ema', '14ema', '21ema',
                         '7macd', '14macd', '7rsi', '14rsi', '21rsi', '7atr', '14atr', '21atr',
                         '7upper', '7lower', '14upper', '14lower', '21upper', '21lower', '7rsv',
                         '14rsv', '21rsv', '3compo_FT', '6compo_FT', '9compo_FT', '27compo_FT',
                         '81compo_FT', '100compo_FT']]

            # define target y = Close, X = rest
            data['y'] = data['Close']
            data = data.drop(columns=['Close'])

            X_all = data.iloc[:, :-1].values   # all features except y
            y_all = data.iloc[:, -1].values    # raw close price (target)

            split_idx = int(0.8 * len(X_all))

            X_train_raw = X_all[:split_idx]
            X_test_raw  = X_all[split_idx:]

            y_train_raw = y_all[:split_idx]
            y_test_raw  = y_all[split_idx:]


            # ==========================================================
            # 2. Fit scalers ONLY on training data
            # ==========================================================
            x_scaler = MinMaxScaler()
            y_scaler = MinMaxScaler()

            X_train_scaled = x_scaler.fit_transform(X_train_raw)
            X_test_scaled  = x_scaler.transform(X_test_raw)

            y_train_scaled = y_scaler.fit_transform(y_train_raw.reshape(-1,1)).flatten()
            y_test_scaled  = y_scaler.transform(y_test_raw.reshape(-1,1)).flatten()


            # ==========================================================
            # 3. Build sliding windows *separately*
            # ==========================================================
            # Training windows
            X_train_seq, y_train_seq = build_sliding_window(X_train_scaled, y_train_scaled, steps)

            # Testing windows
            X_test_seq, y_test_seq = build_sliding_window(X_test_scaled,  y_test_scaled,  steps)

            print("Train windows:", X_train_seq.shape, y_train_seq.shape)
            print("Test windows:",  X_test_seq.shape,  y_test_seq.shape)


            # ==========================================================
            # 4. Use train/test sequences normally
            # ==========================================================
            train_X, train_y = X_train_seq, y_train_seq
            test_X,  test_y  = X_test_seq,  y_test_seq

            train_dataset = TensorDataset(train_X, train_y)
            test_dataset = TensorDataset(test_X, test_y)

            batch_size = 128
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            # -----------------------------
            # Model, loss, optimizer
            # -----------------------------
            num_features = train_X.shape[2]
            model = CNN_BiLSTM_AM_Model(num_features=num_features,
                                        lstm_hidden_size=64,
                                        conv_filters=64).to(device)

            criterion = nn.L1Loss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

            # -----------------------------
            # Training
            # -----------------------------
            epochs = 60
            for epoch in range(epochs):
                model.train()
                total_train_loss = 0.0

                for batch_X, batch_Y in train_loader:
                    batch_X = batch_X.to(device)   # (B, T, F)
                    batch_Y = batch_Y.to(device)   # (B, 1)

                    optimizer.zero_grad()
                    preds = model(batch_X)         # (B, 1)
                    loss = criterion(preds, batch_Y)
                    loss.backward()
                    optimizer.step()

                    total_train_loss += loss.item()

                avg_train_loss = total_train_loss / len(train_loader)
                print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.6f}")

            # -----------------------------
            # Prediction on train & test
            # -----------------------------
            model.eval()
            with torch.no_grad():
                train_pred = model(train_X.to(device)).cpu().numpy()
                test_pred  = model(test_X.to(device)).cpu().numpy()

            # Inverse scaling
            train_true_scaled = train_y.numpy()          # (N_train, 1)
            test_true_scaled  = test_y.numpy()          # (N_test, 1)

            train_true = y_scaler.inverse_transform(train_true_scaled)
            train_pred_inv = y_scaler.inverse_transform(train_pred)

            test_true = y_scaler.inverse_transform(test_true_scaled)
            test_pred_inv = y_scaler.inverse_transform(test_pred)

            # -----------------------------
            # Metrics: RMSE and Accuracy (1 - MAPE)
            # -----------------------------
            RMSE_train = np.sqrt(mean_squared_error(train_true, train_pred_inv))
            RMSE_test = np.sqrt(mean_squared_error(test_true, test_pred_inv))

            acc_train = 1 - np.mean(np.abs((train_pred_inv - train_true) / train_true))
            acc_test = 1 - np.mean(np.abs((test_pred_inv - test_true) / test_true))

            print(f"Train RMSE: {RMSE_train:.4f}, Train Accuracy: {acc_train:.4f}")
            print(f"Test  RMSE: {RMSE_test:.4f}, Test  Accuracy: {acc_test:.4f}")

            re_train_error.append(RMSE_train)
            re_train_acc.append(acc_train)
            re_testing_error.append(RMSE_test)
            re_testing_acc.append(acc_test)

            # -----------------------------
            # Plot predictions
            # -----------------------------
            plt.figure(figsize=(12, 8))
            plt.plot(np.arange(len(train_true)), train_true, color='black', label='Training Actual Price')
            plt.plot(np.arange(len(train_true)), train_pred_inv, color='green', label='Training Predict Price')

            plt.plot(np.arange(len(train_true), len(train_true) + len(test_true)), test_true, color='black', label='Testing Actual Price')
            plt.plot(np.arange(len(train_true), len(train_true) + len(test_true)), test_pred_inv, color='blue', label='Testing Predict Price')

            plt.title(f'{symbol} Testing prediction (seed={seed})')
            plt.ylabel(symbol)
            plt.xlabel('Days (sequence index)')
            plt.legend(loc='upper left')
            plt.show()


        # ==========================================================
        # Save results to CSV
        # ==========================================================
        df = pd.DataFrame({
            #"Seed": seeds,
            "Train Error": re_train_error,
            "Train Accuracy": re_train_acc,
            "Test Error": re_testing_error,
            "Test Accuracy": re_testing_acc
        })

        df.to_csv(f"CNN_BiLSTM_AM_results_{symbol}_{steps}.csv", index=False)
        print("Results saved to:", f"results_{symbol}_CNN_BiLSTM_AM_past{steps}.csv")
        print(df)
